In [1]:
#########################################################################
#Data Collection 
#
# Functions for handling subwatershed characteristics data collection
#
# Authors: Shannon McAvoy (smcavoy@dewberry.com)
#
# Editor: Evan Barnes (ebarnes@dewberry.com)
#
# Copyright: Dewberry Engineers Inc.
#########################################################################

In [2]:
import pandas as pd
import arcpy
import numpy as np
from numpy import mean
from numpy import std
import os
from arcpy import management
# from arcgis.gis import GIS
from time import time
arcpy.CheckOutExtension('Spatial')
# import rasterio

'CheckedOut'

In [3]:
arcpy.env.overwriteOutput = True

In [4]:
path = "C:\\Users\jmatney\Documents\GitHub\IndianaRisk\data\Indiana_ML"
hucs = "C:\\Users\jmatney\Documents\GitHub\IndianaRisk\data\HUCs"

In [5]:
st = time()
# Set Local Variables
output_workspace = os.path.join(path, "working_dsn")

arcpy.env.workspace = output_workspace

##### change these based on what subwatersheds you want #####

#set source for shapefile with all 11 subwatersheds in the county

# all_subwatersheds = r"P:\Temp\McAvoy\ML_DataCollection\Indiana_ML\Indiana_HUC12_groups\1.shp"
all_subwatersheds = os.path.join(hucs, "June_10_HUCS_2.shp")

#set source for streams data
streams = os.path.join(path, "Indiana_50c_streams_edited\Indiana_50c_streams_edited.shp")

#####these are the same for all subwatersheds in Indiana#####

#set source for dem
dem_Indiana = os.path.join(path, "Indiana_3dep\Indiana_dem2")

#set source for slope dem
dem_slope = os.path.join(path, "Indiana_SlopeRaster\IN_slope2")

#set source for NHFL Data
nfhl_sfha = os.path.join(path, "NFHL_18_20200310.gdb\S_FLD_HAZ_AR")

#set source for water bodies data 
water_bodies = os.path.join(path, "IndianaMAP_WaterBodies\Water_Bodies_Lakes_LocalRes\Hydrography_LocalRes_WaterbodyDiscrete_NHD_IN.shp")

#set source for dams data 
dams = os.path.join(path, "IndianaMAP_Dams\Dams_IDNR\Dams_IDNR_IN.shp")

#set source for bridges data
bridges = os.path.join(path, "IndianaMAP_Bridges\Bridges_County_INDOT\Bridges_County_INDOT_IN.shp")

#set source for streets data
streets = os.path.join(path, "IndianaMAP_Streets\Streets_Centerlines_IGIO\County_Street_Centerlines_IGIO_IN.gdb\County_Street_Centerlines_IGIO_IN_Dec2019")

#set source for railraods data
railroads = os.path.join(path, "IndianaMAP_Railroads\Railroads_Active_Abandoned_INDOT\Rail_System_Active_Abandoned_INDOT_IN.shp")

#set source for ACS population data
population = os.path.join(path, "Indiana_PopulationData\Indiana_PopulationData.shp")

#set source for ACS median income data 
median_income = os.path.join(path, "Indiana_IncomeData\Indiana_IncomeData.shp")

#set source for county boundary data
county_boundary = os.path.join(path, "MarionCounty_Boundary\Marion_County_Boundary.shp")

#source for Bing building footprints for Indiana (attrubuted with open street maps data)
building_footprints = os.path.join(path, "C:\\Users\jmatney\Documents\ArcGIS\Projects\Indiana ML Results\Indiana ML Results.gdb\IN_Building_Footprints") #Building_Footprints_Attributed_IN.shp")

#folder with partial duration files, set as workspace temporarily, then reset when done
directory_rainfall = os.path.join(path, "MarionCounty_Rainfall\All_Rainfall_Clipped_IN")

#set source for nlcd land use data
lu_usa = os.path.join(path, r"NLCD_Impervious\NLCD_indiana_polygon.shp")

#set source for impervious indicator data
impervious_usa = os.path.join(path, "NLCD_Impervious\\NLCD_2016_Impervious_L48_20190405_PERCENT\\NLCD_2016_Impervious_L48_20190405.img")


#########################################################################################################
print(round(((time()-st)/60), 2) , 'minutes to process.')

0.01 minutes to process.


In [15]:
subwatershed_list = []
area_list = []
perimeter_list = []
watershed_length_list = []
elongation_ratio_list = []
shape_factor_list = []
circulatory_ratio_list = []
relief_list = []
relief_ratio_list = []
avg_slope_list = []
drainage_density_list = []
ruggedness_list = []
aae_list = []
buildings_aae_list = []
x_list = []
buildings_x_list = []
water_bodies_list = []
dams_list = []
bridges_list = []
streets_list = []
railroads_list = []
population_list = []
dependent_population_list = []
population_density_list = []
avg_median_income_list = []
housing_density_list = []
population_change_list = []
dist_to_stream_avg_list = []
dist_to_stream_stdev_list = []

lu_21_list = []
lu_22_list = []
lu_23_list = []
lu_24_list = []
lu_41_list = []
lu_82_list = []
impervious_percent_list = []

orb100yr06h_list = []
orb100yr12h_list = []
orb100yr24h_list = []
orb25yr06h_list = []
orb25yr12h_list = []
orb25yr24h_list = []
orb2yr06h_list = []
orb2yr12h_list = []
orb2yr24h_list = []
orb50yr06h_list = []
orb50yr12h_list = []
orb50yr24h_list = []
orb100yr06ha_am_list = []
orb100yr12ha_am_list = []
orb100yr24ha_am_list = []
orb25yr06ha_am_list = []
orb25yr12ha_am_list = []
orb25yr24ha_am_list = []
orb2yr06ha_am_list = []
orb2yr12ha_am_list = []
orb2yr24ha_am_list = []
orb50yr06ha_am_list = []
orb50yr12ha_am_list = []
orb50yr24ha_am_list = []

In [16]:
print(all_subwatersheds)

C:\Users\jmatney\Documents\GitHub\IndianaRisk\data\HUCs\June_10_HUCS_2.shp


In [8]:
#add AREA and PERIMETER fields for the whole subwatersheds file
#calculate area of subwatersheds
arcpy.AddField_management(all_subwatersheds, "AREA", "DOUBLE")

arcpy.CalculateGeometryAttributes_management(all_subwatersheds, "AREA AREA_GEODESIC", '', 
                                                    "SQUARE_KILOMETERS",
                                                    None)
#calculate perimeter of subwatershed
arcpy.AddField_management(all_subwatersheds, "PERIMETER", "DOUBLE")
perimeter = arcpy.CalculateGeometryAttributes_management(all_subwatersheds, "PERIMETER PERIMETER_LENGTH_GEODESIC",
                                                         "KILOMETERS")

print("perimeter calculated")

perimeter calculated


In [9]:
st = time()
# perform all need intersections here:

#put all intersections in this projection so area and length will be in meters
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(102008)

water_bodies_intersect = "water_bodies.shp"
water_bodies_intersect = arcpy.Intersect_analysis([water_bodies, all_subwatersheds], water_bodies_intersect)

population_intersect = "population.shp"
arcpy.Intersect_analysis([population, all_subwatersheds], population_intersect)

median_income_intersect= "median_income.shp"
arcpy.Intersect_analysis([median_income, all_subwatersheds], median_income_intersect)

# building_footprints_intersect = "building_footprints.shp"
# arcpy.Intersect_analysis([building_footprints, all_subwatersheds], building_footprints_intersect)

streams_intersect = "streams.shp"
arcpy.Intersect_analysis([streams, all_subwatersheds], streams_intersect)


print(round(((time()-st)/60), 2) , 'minutes to process.')

3.02 minutes to process.


In [10]:
# building_footprints_intersect = "building_footprints.shp"
# arcpy.Intersect_analysis([building_footprints, all_subwatersheds], building_footprints_intersect)

In [11]:
def get_length_area(layer, length_area):
    #length_Area = 'LENGTH' or 'AREA'
    search_field = "SHAPE@{}".format(length_area)
    values = []
    with arcpy.da.SearchCursor(layer, [search_field]) as sCursor: 
        for s in sCursor:
            # convert meters to kilometers
            if length_area == 'AREA':
                value = s[0]/ (1000 * 1000)
            else: # assumed length
                value = s[0] / 1000
                    
            values.append(value)
    return values

In [17]:
st = time()
# make all layers for selections
#subwatershed_selection = "subwatershed_lyr"
#arcpy.MakeFeatureLayer_management (all_subwatersheds, subwatershed_selection)

# nfhl_sfha_selection = "nfhl_sfha_lyr"
# arcpy.MakeFeatureLayer_management(nfhl_sfha_intersect, nfhl_sfha_selection)

water_bodies_selection = "water_bodies_lyr"
arcpy.MakeFeatureLayer_management(water_bodies_intersect, water_bodies_selection)

# dams_selection = "dams_lyr"
# arcpy.MakeFeatureLayer_management(dams_intersect, dams_selection)

# bridges_selection = "bridges_lyr"
# arcpy.MakeFeatureLayer_management(bridges_intersect, bridges_selection)

# streets_selection = "streets_lyr"
# arcpy.MakeFeatureLayer_management(streets_intersect, streets_selection)

# railroads_selection = "railroads_lyr"
# arcpy.MakeFeatureLayer_management(railroads_intersect, railroads_selection)

population_selection = "population_lyr"
arcpy.MakeFeatureLayer_management(population_intersect, population_selection)

median_income_selection = "median_income_lyr"
arcpy.MakeFeatureLayer_management(median_income_intersect, median_income_selection)

# building_footprints_selection= "building_footprints_lyr"
# arcpy.MakeFeatureLayer_management(building_footprints_intersect, building_footprints_selection)

streams_selection = "streams_lyr"
arcpy.MakeFeatureLayer_management(streams_intersect, streams_selection)

# lu_selection = "lu_lyr"
# arcpy.MakeFeatureLayer_management(lu_subwatershed, lu_selection)

print(round(((time()-st)/60), 2) , 'minutes to process.')

0.01 minutes to process.


In [19]:
st = time()


#search cursor through each row of county subwatersheds file
with arcpy.da.SearchCursor(all_subwatersheds, ['HUC12','AREA', 'PERIMETER']) as cursor:

    for row in cursor:
        if row[0] in ['051202081004',  
         '051402011005',  
         '051402020603', 
         '051202081401',  
         '051402011202', 
         '051201130709',  
         '051402011103', 
         '051201130901', 
         '051402010802',  
         '051402010801',
         '051402011006',  
         '051201130903',
         '051201130707',
         '051402011204',   
         '051402020604',
         '040500010802',
         '051402011102']:
            next(cursor) # Does nothing, skips next item
        else:
            subwatershed_number = row[0]

            subwatershed_list.append(subwatershed_number)

            print("--------------------", subwatershed_number, " is the subwatershed to be worked on ------------------------")
            # Make a layer from the feature class
            arcpy.MakeFeatureLayer_management(all_subwatersheds,"subwatersheds_lyr")

            #select subwatershed from shapefile with all of them
            s = arcpy.SelectLayerByAttribute_management(all_subwatersheds, "NEW_SELECTION", 
                                                    "HUC12 = '{}'".format(subwatershed_number))
            arcpy.CopyFeatures_management(s, "s_lyr.shp")
            subwatershed_selection1 = os.path.join(path, "working_dsn\s_lyr.shp")
            #arcpy.MakeFeatureLayer_management(s, subwatershed_selection1)

            # use mask in lieu of clipping rasters
            arcpy.env.cellSize = dem_slope
            arcpy.env.snapRaster = dem_slope
            #arcpy.env.outputCoordinateSystem = 
            arcpy.env.mask = subwatershed_selection1

            area = row[1]
            area_list.append(area)
            print(area, " square kilometers")

            perimeter = row[2]
            perimeter_list.append(perimeter)
            print(perimeter, " kilometers")


            #calculate area covered by lakes/reserviors 
            arcpy.management.SelectLayerByAttribute(water_bodies_selection, "NEW_SELECTION", 
                                                                "HUC12 = '{}'".format(subwatershed_number))
            water_bodies_area_sum = sum(get_length_area(water_bodies_selection, "AREA"))
            water_bodies_list.append(water_bodies_area_sum)
            print(water_bodies_area_sum, " = area of all water bodies in the subwatershed (square km)")


            #all streams in subwatershed
            arcpy.management.SelectLayerByAttribute(streams_selection, "NEW_SELECTION", 
                                                                "HUC12 = '{}'".format(subwatershed_number))
            print("streams clipped")
            print("length calculated")
            #get the sum of all of the areas
            stream_length_list = get_length_area(streams_selection, "LENGTH")  
            stream_length_sum = sum(stream_length_list)
            print(stream_length_sum, " = length of all streams in subwatershed")
            print(area, " = subwatershed area")
            drainage_density = stream_length_sum / area
            drainage_density_list.append(drainage_density)
            print(drainage_density, "is the drainage density (streams/km)")
            #find watershed length
            #watershed length = distance from outlet to watershed boundary along the main channel
            #we are assuming that the longest stream above is the main channel
            watershed_length = max(stream_length_list)
            watershed_length_list.append(watershed_length)
            print(watershed_length, " = watershed length (longest stream in subwatershed)")
            #calculate shape factor 
            #watershed length squared divided by watershed area
            shape_factor = (watershed_length**2) / area
            shape_factor_list.append(shape_factor)
            print(shape_factor, " is the shape factor.")


            #use Raster Domain tool to get z-enabled polyline of perimeter
            dem_clip = os.path.join(path, "working_dsn\dem_clip.tif")
            arcpy.Clip_management(dem_Indiana, "#", dem_clip, subwatershed_selection1, "#", "ClippingGeometry", "NO_MAINTAIN_EXTENT")
            perimeter_polyline = "perimeter_polyline.shp"
            arcpy.RasterDomain_3d(dem_clip, perimeter_polyline, "LINE")
            print("created 3d polyline of subwatershed perimeter")

            #Convert each vertices of the polyline into points
            perimeter_points = "perimeter_points.shp"
            arcpy.FeatureVerticesToPoints_management(perimeter_polyline, perimeter_points)
            print("created perimeter points")

            #get Z values into the attribute table for the points
            arcpy.AddZInformation_3d(perimeter_points, "Z")
            print("added z information")

            #get the max value from all of the points
            perimeter_stats = "perimeter_stats"
            arcpy.Statistics_analysis(perimeter_points, "perimeter_stats", [["Z", "MAX"], ["Z", "MIN"]])

            #get the values for max Z and min Z into a format to use them
            with arcpy.da.SearchCursor(perimeter_stats, ['MAX_Z', 'MIN_Z']) as statsCursor:
                for stat in statsCursor:
                    max_z = stat[0]
                    min_z = stat[1]


            print(max_z)
            print(min_z)

            relief = max_z - min_z

            #calculate relief ratio
            #relief divided by watershed length
            #length is is kilometers, convert to meters
            watershed_length_meters = watershed_length * 1000
            relief_ratio = relief / watershed_length_meters
            relief_ratio_list.append(relief_ratio)
            print(relief_ratio, " = relief ratio")


            #calculate ruggedness number
            #product of relief and drainage density
            #relief is in meters, convert first to km
            relief_km = relief / 1000
            print(relief_km, " is the relief in km")
            ruggedness = relief_km * drainage_density
            ruggedness_list.append(ruggedness)
            print(ruggedness, " is the ruggedness number")


                    #ACS population data - 5 year estimates (2014-2018) gotten from ESRI Living Atlas Data
            #data is chosen to be on census tract level
            arcpy.management.SelectLayerByAttribute(population_selection, "NEW_SELECTION", 
                                                                "HUC12 = '{}'".format(subwatershed_number))
            #get total population
            #field = B01001_001E = Total Population (alias)
            #when clipped, field name changes to B01001_001

            #get the sum of all of the populations in each tract
            total_pop_list = []  
            with arcpy.da.SearchCursor(population_selection, ['B01001_001']) as popCursor:  
                for pop in popCursor:  
                    total_pop_value = pop[0] 
                    total_pop_list.append(total_pop_value)  
            total_pop_sum = sum(total_pop_list)
            population_list.append(total_pop_sum)
            print(total_pop_sum, " = total population in the subwatershed") 

            #get the average of percentages of dependent age groups in each tract
            #field = B01001_calc_pctDependE = Percent of Population in Dependent Age Groups (under 18 and 65+) (alias)
            #when clipped field changes to B01001_61
            dependent_pop_list = []  
            with arcpy.da.SearchCursor(population_selection, ["B01001__61"]) as dpopCursor: 
                for dpop in dpopCursor:  
                    dependent_pop_value = dpop[0] 
                    dependent_pop_list.append(dependent_pop_value)  
            dependent_pop_avg_pct = mean(dependent_pop_list)
            dependent_population_list.append(dependent_pop_avg_pct)

            print(dependent_pop_avg_pct, " = total percent of dependent population in the subwatershed") 

            #find population density
            population_density = total_pop_sum / area
            population_density_list.append(population_density)
            print(population_density, " = population density of subwatershed (people/square km)")


            #get housing density
            #select buildings that are marked residential
    #         arcpy.SelectLayerByAttribute_management(building_footprints_selection, "NEW_SELECTION",
    #                                                                              "HUC12 = '{}' And RES_NONRES = 'Res'".format(subwatershed_number))
    #         #get count of how many buildings there are 
    #         buildings_count = arcpy.GetCount_management(building_footprints_selection)
    #         buildings_count_number = buildings_count.getOutput(0)
    #         print(buildings_count_number, " = number of residential building footprints in the subwatershed")
    #         #divide number of buildings by subwatershed area
    #         housing_density = int(buildings_count_number) / area
    #         housing_density_list.append(housing_density)
    #         print(housing_density, " = housing density (buildings per square km)")



        #########################################################################################




    print(round(((time()-st)/60), 2) , 'minutes to process.')    


-------------------- 051201111702  is the subwatershed to be worked on ------------------------
61.8773909597  square kilometers
51.7534192078  kilometers
0.18462783320343096  = area of all water bodies in the subwatershed (square km)
streams clipped
length calculated
16.551041910000606  = length of all streams in subwatershed
61.8773909597  = subwatershed area
0.26748125047450855 is the drainage density (streams/km)
9.160542170246627  = watershed length (longest stream in subwatershed)
1.3561582276072364  is the shape factor.
created 3d polyline of subwatershed perimeter
created perimeter points
added z information
161.37588501
124.493553162
0.004026217134592048  = relief ratio
0.036882331847999994  is the relief in km
0.00986533224311883  is the ruggedness number
12975  = total population in the subwatershed
33.06666666666667  = total percent of dependent population in the subwatershed
209.6888669473873  = population density of subwatershed (people/square km)
-------------------- 051

created 3d polyline of subwatershed perimeter
created perimeter points
added z information
238.30696106
168.000747681
0.005017117709498193  = relief ratio
0.07030621337899999  is the relief in km
0.023431163219059484  is the ruggedness number
16455  = total population in the subwatershed
39.13333333333333  = total percent of dependent population in the subwatershed
189.7355079162027  = population density of subwatershed (people/square km)
-------------------- 051401010803  is the subwatershed to be worked on ------------------------
40.9359721686  square kilometers
34.1197883757  kilometers
0.5707071741880579  = area of all water bodies in the subwatershed (square km)
streams clipped
length calculated
67.78919002889818  = length of all streams in subwatershed
40.9359721686  = subwatershed area
1.6559809487289023 is the drainage density (streams/km)
11.724735289793868  = watershed length (longest stream in subwatershed)
3.358156905363147  is the shape factor.
created 3d polyline of subw

98.2789774591  square kilometers
73.3017173737  kilometers
4.741852499953684  = area of all water bodies in the subwatershed (square km)
streams clipped
length calculated
42.7098079653337  = length of all streams in subwatershed
98.2789774591  = subwatershed area
0.4345772521199451 is the drainage density (streams/km)
10.470189018854581  = watershed length (longest stream in subwatershed)
1.1154456520079767  is the shape factor.
created 3d polyline of subwatershed perimeter
created perimeter points
added z information
261.75692749
209.198867798
0.0050197813618602415  = relief ratio
0.052558059692  is the relief in km
0.02284053715770541  is the ruggedness number
14298  = total population in the subwatershed
42.974999999999994  = total percent of dependent population in the subwatershed
145.48380914881096  = population density of subwatershed (people/square km)
-------------------- 051402020204  is the subwatershed to be worked on ------------------------
90.3266891126  square kilometer

created perimeter points
added z information
273.712005615
238.047134399
0.002013020882206996  = relief ratio
0.035664871216000024  is the relief in km
0.01540733710121677  is the ruggedness number
8713  = total population in the subwatershed
42.900000000000006  = total percent of dependent population in the subwatershed
108.82406068158133  = population density of subwatershed (people/square km)
-------------------- 041000040604  is the subwatershed to be worked on ------------------------
51.2184193214  square kilometers
48.5216920093  kilometers
0.7747033656655598  = area of all water bodies in the subwatershed (square km)
streams clipped
length calculated
22.641918772238096  = length of all streams in subwatershed
51.2184193214  = subwatershed area
0.4420659417495511 is the drainage density (streams/km)
12.33560603796557  = watershed length (longest stream in subwatershed)
2.9709463575794968  is the shape factor.
created 3d polyline of subwatershed perimeter
created perimeter points

65.7745845298  square kilometers
48.1997129362  kilometers
0.4800839557203092  = area of all water bodies in the subwatershed (square km)
streams clipped
length calculated
46.532670823222325  = length of all streams in subwatershed
65.7745845298  = subwatershed area
0.7074567046811238 is the drainage density (streams/km)
12.814819514856556  = watershed length (longest stream in subwatershed)
2.4967029495100297  is the shape factor.
created 3d polyline of subwatershed perimeter
created perimeter points
added z information
323.159454346
265.087738037
0.004531606258026182  = relief ratio
0.05807171630900001  is the relief in km
0.04108322505514222  is the ruggedness number
12656  = total population in the subwatershed
42.9  = total percent of dependent population in the subwatershed
192.41474637161778  = population density of subwatershed (people/square km)
-------------------- 051202070302  is the subwatershed to be worked on ------------------------
60.3765740263  square kilometers
37.5

...

59.4668879068  square kilometers
51.9410845147  kilometers
0.14526450169959274  = area of all water bodies in the subwatershed (square km)
streams clipped
length calculated
52.19470617154354  = length of all streams in subwatershed
59.4668879068  = subwatershed area
0.8777104033650819 is the drainage density (streams/km)
19.06722452774335  = watershed length (longest stream in subwatershed)
6.113638429526844  is the shape factor.
created 3d polyline of subwatershed perimeter
created perimeter points
added z information
256.494293213
228.61630249
0.001462089602104214  = relief ratio
0.02787799072299998  is the relief in km
0.024468802482492325  is the ruggedness number
10720  = total population in the subwatershed
40.9  = total percent of dependent population in the subwatershed
180.2683876244039  = population density of subwatershed (people/square km)
-------------------- 051202060405  is the subwatershed to be worked on ------------------------
91.6560179872  square kilometers
65.2167

created perimeter points
added z information
298.613189697
258.666992188
0.0016360652710624701  = relief ratio
0.039946197509  is the relief in km
0.022561413404114786  is the ruggedness number
23159  = total population in the subwatershed
40.150000000000006  = total percent of dependent population in the subwatershed
349.3789688004646  = population density of subwatershed (people/square km)
-------------------- 051202070801  is the subwatershed to be worked on ------------------------
58.780843313  square kilometers
47.1058487435  kilometers
0.8823587496553341  = area of all water bodies in the subwatershed (square km)
streams clipped
length calculated
61.49134013954742  = length of all streams in subwatershed
58.780843313  = subwatershed area
1.0461119077879573 is the drainage density (streams/km)
17.702999153863626  = watershed length (longest stream in subwatershed)
5.3316039950788765  is the shape factor.
created 3d polyline of subwatershed perimeter
created perimeter points
added

81.000347716  square kilometers
53.7534826494  kilometers
0.32247426030258486  = area of all water bodies in the subwatershed (square km)
streams clipped
length calculated
47.924678306202466  = length of all streams in subwatershed
81.000347716  = subwatershed area
0.5916601552654311 is the drainage density (streams/km)
30.01503181905943  = watershed length (longest stream in subwatershed)
11.12220083619709  is the shape factor.
created 3d polyline of subwatershed perimeter
created perimeter points
added z information
233.576583862
147.959350586
0.0028524785111716387  = relief ratio
0.085617233276  is the relief in km
0.0506563055334748  is the ruggedness number
3739  = total population in the subwatershed
42.6  = total percent of dependent population in the subwatershed
46.16029567069915  = population density of subwatershed (people/square km)
-------------------- 050800030706  is the subwatershed to be worked on ------------------------
41.671318725  square kilometers
34.5750241485  

created perimeter points
added z information
308.143371582
264.847076416
0.0028764975853062423  = relief ratio
0.04329629516599999  is the relief in km
0.022871574491494604  is the ruggedness number
12360  = total population in the subwatershed
40.4  = total percent of dependent population in the subwatershed
165.1373158631687  = population density of subwatershed (people/square km)
-------------------- 040400010508  is the subwatershed to be worked on ------------------------
49.18290844  square kilometers
43.2010382729  kilometers
1.428799570316155  = area of all water bodies in the subwatershed (square km)
streams clipped
length calculated
19.237757087111376  = length of all streams in subwatershed
49.18290844  = subwatershed area
0.39114720331312264 is the drainage density (streams/km)
12.374736993845005  = watershed length (longest stream in subwatershed)
3.1135636448513395  is the shape factor.
created 3d polyline of subwatershed perimeter
created perimeter points
added z informa

72.5162611525  square kilometers
51.4523267834  kilometers
0.16868888091004744  = area of all water bodies in the subwatershed (square km)
streams clipped
length calculated
43.4525191341683  = length of all streams in subwatershed
72.5162611525  = subwatershed area
0.5992106934855435 is the drainage density (streams/km)
20.567693373857068  = watershed length (longest stream in subwatershed)
5.83358827382705  is the shape factor.
created 3d polyline of subwatershed perimeter
created perimeter points
added z information
284.09588623
225.124694825
0.0028671757368746264  = relief ratio
0.05897119140500001  is the relief in km
0.03533616849745878  is the ruggedness number
21605  = total population in the subwatershed
40.6  = total percent of dependent population in the subwatershed
297.9331760439937  = population density of subwatershed (people/square km)
-------------------- 051202011004  is the subwatershed to be worked on ------------------------
57.1526244752  square kilometers
50.71345

In [22]:
st = time()
outputs = {'subwatershed': subwatershed_list,
           'drainage_density': drainage_density_list,
           'shape_factor': shape_factor_list,
           'ruggedness': ruggedness_list,
           'population_density': population_density_list
          }
outputs_df = pd.DataFrame(outputs, columns = ['subwatershed',
                                              'drainage_density',
                     
                                              'shape_factor',
                                              'ruggedness',
                                              'population_density'
                                             ])    
    

print(outputs_df)

outputs_df.to_excel(os.path.join(path,"HOTFIX\\fixed_2.xlsx"))
print(round(((time()-st)/60), 2) , 'minutes to process.')

     subwatershed  drainage_density  shape_factor  ruggedness  \
0    051201111702          0.267481      1.356158    0.009865   
1    051202060303          0.499391      9.027316    0.049596   
2    051202011206          0.853749     11.680504    0.064245   
3    051201030604          0.383356      0.960323    0.012636   
4    051201011403          0.553784      2.579482    0.037767   
..            ...               ...           ...         ...   
641  051202011104          0.417816      4.505143    0.017056   
642  051201060204          0.247740      1.950915    0.015124   
643  071200030303          0.246145      2.537285    0.007595   
644  051202030805          0.498546      1.892778    0.026726   
645  051201111502          0.530830      8.542286    0.033382   

     population_density  
0            209.688867  
1            199.537015  
2           2474.883564  
3            301.468981  
4            164.427085  
..                  ...  
641          997.191314  
642        

In [21]:
len(population_density_list)

646

In [23]:
path

'C:\\Users\\jmatney\\Documents\\GitHub\\IndianaRisk\\data\\Indiana_ML'

In [ ]:
len(outputs['population_density'])